In [1]:
'''A chatbot based on sklearn where you can give a symptom and  it will ask you questions and will tell you the details and give some advice.'''

'A chatbot based on sklearn where you can give a symptom and  it will ask you questions and will tell you the details and give some advice.'

In [1]:
#importing all Relavent libraries
import pandas as pd
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv #Comma Separated Values
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#Import Data from csv file
training = pd.read_csv('Training.csv')#read_csv is to read csv files and do operations on it
testing= pd.read_csv('Testing.csv')
cols= training.columns
cols= cols[:-1] #starts from the end towards the first taking each element
x = training[cols]
y = training['prognosis']
y1= y
#Group records by the prognosis 
reduced_data = training.groupby(training['prognosis']).max()


In [3]:
#mapping strings to numbers
# Encode labels in column species. Label encoding convert the data in machine readable form,
#but it assigns a unique number(starting from 0) to each class of data
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

# Spliting our dataset into training and testing subsets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']  
testy    = le.transform(testy) #Label encoder 


clf1  = DecisionTreeClassifier() 
clf = clf1.fit(x_train,y_train) #To fit algorithm to training data
scores = cross_val_score(clf, x_test, y_test, cv=3)
print (scores.mean())

model=SVC() #designed to bring the system closer to unity power factor
model.fit(x_train,y_train)#To fit algorithm to training data
print(model.score(x_test,y_test))#measuring the accuracy of the model against the training data

0.9664634146341463
1.0


In [4]:
importances = clf.feature_importances_
#indices is an array of indices returned from your np.argsort(importances)[::-1]
indices = np.argsort(importances)[::-1]
features = cols

In [5]:
severityDictionary=dict() #function creates a dictionary
description_list = dict()
precautionDictionary=dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index
 
 # define function for calculation
def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor. ")
    else:
        print("It might not be that bad but you should take precautions.")


def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            description={row[0]:row[1]} 
            description_list.update(description)


def getSeverityDict():
    global severityDictionary
    with open('symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                diction={row[0]:int(row[1])}
                severityDictionary.update(diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(prec)

# to Get input from user
def getInfo():
    print("Enter Your Name \n\t\t\t\t\t\t",end="   ")
    name=input("")
    print("hello ",name)

def check_pattern(dis_list,inp): #Pass arguments as list and input
    import re
    pred_list=[]
    ptr=0
    patt = "^" + inp + "$"
    regex = re.compile(inp) #re.compile() method is used to compile our input pattern 
    #provided as a string into a regex pattern object . 
    #Later we can use this pattern object to search for a match inside different target strings
    for item in dis_list:

        
        if regex.search(item):
            pred_list.append(item)
            
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return ptr,item
    
def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv') #Reading data from csv file to perform operations
    X = df.iloc[:, :-1]
    y = df['prognosis']
    
# Spliting our dataset into training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train) #to fit algorithm on training data 

    symptoms_dict = {} #creat an empty dictionary

    for index, symptom in enumerate(X): #Enumerate() method adds a counter to an iterable
                                        #and returns it in a form of enumerate object
        symptoms_dict[symptom] = index

    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[[symptoms_dict[item]]] = 1


    return rf_clf.predict([input_vector])


def print_disease(node):
    node = node[0]
    val  = node.nonzero() 
    disease = le.inverse_transform(val[0])
    return disease


In [6]:
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    chk_dis=",".join(feature_names).split(",") # joins strings with splited by comma
    
    
    symptoms_present = [] # Create empty list 
    while True:
        
      #Taking input from user
        print("Enter the symptom you are experiencing  \n\t\t\t\t\t\t",end="->")
        disease_input = input("")
        conf,cnf_dis=check_pattern(chk_dis,disease_input)
        #Checking for conditional statement
        if conf==1:
            print("searches related to input: ")
            for num,it in enumerate(cnf_dis):
                print(num,")",it)
            if num!=0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                conf_inp = int(input(""))
            else:
                conf_inp=0

            disease_input=cnf_dis[conf_inp]
            break
            
        else:
            print("Enter valid symptom.")

    while True:
        try:
            num_days=int(input("Okay. From how many days ? : "))
            break
        except:
            print("Enter number of days.")
            
    
    # Defining function 
    def recurse(node, depth): 
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            if name == disease_input:
                val = 1
            else:
                val = 0
            if  val <= threshold:
                 recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
                present_disease = print_disease(tree_.value[node])

                red_cols = reduced_data.columns 
                symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]
                #loc takes index label and returns row and dataframe

                print("Are you experiencing any ")
                symptoms_exp=[]
                for syms in list(symptoms_given):
                    inp=""
                    print(syms,"? : ",end='')
                    while True:
                        inp=input("")
                        if(inp=="yes" or inp=="no"):
                            break
                        else:
                            print("provide proper answers i.e. (yes/no) : ",end="")
                    if(inp=="yes"):
                        symptoms_exp.append(syms)

                second_prediction=sec_predict(symptoms_exp)

                calc_condition(symptoms_exp,num_days)
                if(present_disease[0]==second_prediction[0]):
                    print("You may have ", present_disease[0])

                    print(description_list[present_disease[0]])



                else:
                    print("You may have ", present_disease[0], "or ", second_prediction[0])
                    print(description_list[present_disease[0]])
                    print(description_list[second_prediction[0]])

                precution_list=precautionDictionary[present_disease[0]]
                print("Take following measures : ")
                for  i,j in enumerate(precution_list):
                    print(i+1,")",j)
                

    recurse(0, 1) #Call recurse function


In [8]:
#Calling the functions
getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf,cols)

Enter Your Name 
						   Sakshi Tripathi
hello  Sakshi Tripathi
Enter the symptom you are experiencing  
						->headache
searches related to input: 
0 ) headache
Okay. From how many days ? : 3
Are you experiencing any 
muscle_weakness ? : yes
stiff_neck ? : no
swelling_joints ? : no
movement_stiffness ? : no
painful_walking ? : no
It might not be that bad but you should take precautions.
You may have  Arthritis or  (vertigo) Paroymsal  Positional Vertigo
Arthritis is the swelling and tenderness of one or more of your joints. The main symptoms of arthritis are joint pain and stiffness, which typically worsen with age. The most common types of arthritis are osteoarthritis and rheumatoid arthritis.
Benign paroxysmal positional vertigo (BPPV) is one of the most common causes of vertigo â€” the sudden sensation that you're spinning or that the inside of your head is spinning. Benign paroxysmal positional vertigo causes brief episodes of mild to intense dizziness.
Take following measures :